### SQL Databases

In [1]:
## Create sample SQLite Database
import sqlite3
import os

os.makedirs("data/databases", exist_ok=True) #exist_ok=True if the file is there no need to create again and vice versa if it is false


In [2]:
#Create sample database
conn = sqlite3.connect("data/databases/company.db")
cursor = conn.cursor()


In [3]:
#Create tables
cursor.execute("""CREATE TABLE IF NOT EXISTS employees
(id INTEGER PRIMARY KEY, name TEXT, role TEXT, department TEXT, salary REAL)""")

In [4]:
cursor.execute("""CREATE TABLE IF NOT EXISTS projects
(id INTEGER PRIMARY KEY, name TEXT, status TEXT, budget TEXT, lead_id INTEGER)""")

In [5]:
employees = [
    (1, 'John Doe', 'Senior Developer', 'Engineering', 95000),
    (2, 'Jane Smith', 'Data Scientist', 'Analytics', 105000),
    (3, 'Mike Johnson', 'Product Manager', 'Product', 110000),
    (4, 'Sarah Williams', 'DevOps Engineer', 'Engineering', 98000)
]

projects = [
    (1, 'RAG Implementation', 'Active', 150000, 1),
    (2, 'Data Pipeline', 'Completed', 80000, 2),
    (3, 'Customer Portal', 'Planning', 200000, 3),
    (4, 'ML Platform', 'Active', 250000, 2)
]

In [6]:
cursor.executemany('INSERT OR REPLACE INTO employees VALUES (?,?,?,?,?)', employees)

In [7]:
cursor.executemany('INSERT OR REPLACE INTO projects VALUES (?,?,?,?,?)', projects)

In [8]:
cursor.execute("select * from employees")

In [9]:
conn.commit()
conn.close()

### Database Content Extraction

In [10]:
from langchain_community.utilities import SQLDatabase
from langchain_community.document_loaders import SQLDatabaseLoader

In [11]:
# Method 1 SQLDATABASE Utility
db = SQLDatabase.from_uri("sqlite:///data/databases/company.db")

## get Database info
print(f"Tables: {db.get_usable_table_names()}")
print(f"\nTable DDL :")
print(db.get_table_info())

Tables: ['employees', 'projects']

Table DDL :

CREATE TABLE employees (
	id INTEGER, 
	name TEXT, 
	role TEXT, 
	department TEXT, 
	salary REAL, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	role	department	salary
1	John Doe	Senior Developer	Engineering	95000.0
2	Jane Smith	Data Scientist	Analytics	105000.0
3	Mike Johnson	Product Manager	Product	110000.0
*/


CREATE TABLE projects (
	id INTEGER, 
	name TEXT, 
	status TEXT, 
	budget REAL, 
	lead_id INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from projects table:
id	name	status	budget	lead_id
1	RAG Implementation	Active	150000.0	1
2	Data Pipeline	Completed	80000.0	2
3	Customer Portal	Planning	200000.0	3
*/


In [12]:
from typing import List
from langchain_core.documents import Document
# Method 2: Custom SQL to Document conversion
print("\n Custom SQL Processing")
def sql_to_documents(db_path:str) -> List[Document]:
    """Convert SQL Database to SQL Database To Documents with context"""
    conn = sqlite3.connect(db_path)
    cursor=conn.cursor()
    documents = []          
    # Strategy 1 : Create documents for each table
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")     
    tables = cursor.fetchall()

    for table in tables:
        table_name=table[0]

        #Get Table Schema
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        column_names = [col[1] for col in columns]

        #Get Table data
        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()

        #Create table data
        cursor.execute(f"SELECT * from {table_name}")
        rows = cursor.fetchall()

        #Create table overview document
        table_content = f"Table : {table_name}\n"
        table_content += f"Columns :{','.join(column_names)}\n"
        table_content += f"Total Records: {len(rows)}\n\n"

        #Add Sample records
        table_content += "Sample Records :\n"
        for row in rows[:]:  #First 5 Records
            record = dict(zip(column_names,row))
            table_content += f"{record}\n"

        doc = Document(
            page_content=table_content,
            metadata={
                'source' : db_path,
                'table_name' : table_name,
                "num_records" : len(rows),
                'data_type' : 'sql_table'
            }
        )
        documents.append(doc)


 Custom SQL Processing


In [13]:
sql_to_documents("data/databases/company.db")
